# Pre-processing News Data

## Bài toán
Dữ liệu gồm n văn bản phân vào 10 chủ đề khác nhau. Cần biễu diễn mỗi văn bản dưới dạng một vector số thể hiện cho nội dụng của văn bản đó.

## Mục lục
- Load dữ liệu từ thư mục
- Loại bỏ các stop words
- Sử dụng thư viện để mã hóa TF-IDF cho mỗi văn bản

## Phương pháp mã hóa: TF-IDF
Cho tập gồm $n$ văn bản: $D = \{d_1, d_2, ... d_n\}$. Tập từ điển tương ứng được xây dựng từ $n$ văn bản này có độ dài là $m$
- Xét văn bản $d$ có $|d|$ từ và $t$ là một từ trong $d$. Mã hóa tf-idf của $t$ trong văn bản $d$ được biểu diễn:
\begin{equation}
    \begin{split}
        \text{tf}_{t, d} &= \frac{f_t}{|d|} \\
        \text{idf}_{t, d} &= \log\frac{n}{n_t}, \ \ \ \ n_t = |\{d\in D: t\in d\}| \\
        \text{tf-idf}_{t d} &= \text{tf}_{t, d} \times \text{idf}_{t, d}
    \end{split}
\end{equation}

- Khi đó văn bản $d$ được mã hóa là một vector $m$ chiều. Các từ xuất hiện trong d sẽ được thay bằng giá trị tf-idf tương ứng. Các từ không xuất hiện trong $d$ thì thay là 0

In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_files
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

import re
from unidecode import unidecode
%matplotlib inline

## Load dữ liệu từ thư mục

Cấu trúc thư mục như sau 

- data/news_vnexpress/

    - Kinh tế: 
        - bài báo 1.txt 
        - bài báo 2.txt 
    - Pháp luật
        - bài báo 3.txt 
        - bài báo 4.txt 

In [11]:
INPUT = 'news_vnexpress'
os.makedirs("images",exist_ok=True)  

In [ ]:
print('Các nhãn và số văn bản tương ứng trong dữ liệu')
print('----------------------------------------------')
n = 0
for label in os.listdir(INPUT):
    if os.path.isdir(os.path.join(INPUT, label)):  # Kiểm tra đường dẫn có phải là một thư mục
        print(f'{label}: {len(os.listdir(os.path.join(INPUT, label)))}')
        n += len(os.listdir(os.path.join(INPUT, label)))

print('-------------------------')
print(f"Tổng số văn bản: {n}")

Các nhãn và số văn bản tương ứng trong dữ liệu
----------------------------------------------
doi-song: 120
du-lich: 54
giai-tri: 201
giao-duc: 105
khoa-hoc: 144
kinh-doanh: 262
phap-luat: 59
suc-khoe: 162
the-thao: 173
thoi-su: 59
-------------------------
Tổng số văn bản: 1339


In [5]:
# load data
data_train = load_files(container_path=INPUT, encoding="utf-8")
print('mapping:')
for i in range(len(data_train.target_names)):
    print(f'{data_train.target_names[i]} - {i}')

print('--------------------------')
print(data_train.filenames[0:5])
# print(data_train.data[0:1])
print(data_train.target[0:5])
print(data_train.data[0:4])

print("\nTổng số  văn bản: {}" .format( len(data_train.filenames)))

mapping:
doi-song - 0
du-lich - 1
giai-tri - 2
giao-duc - 3
khoa-hoc - 4
kinh-doanh - 5
phap-luat - 6
suc-khoe - 7
the-thao - 8
thoi-su - 9
--------------------------
['news_vnexpress\\khoa-hoc\\00133.txt'
 'news_vnexpress\\suc-khoe\\00102.txt'
 'news_vnexpress\\kinh-doanh\\00240.txt'
 'news_vnexpress\\doi-song\\00085.txt'
 'news_vnexpress\\kinh-doanh\\00123.txt']
[4 7 5 0 5]
['Mời độc giả đặt câu hỏi tại đây\n', '(Nguồn và ảnh: Trung tâm cấy tóc New Hair)\n', 'Lợi suất trái phiếu chính phủ Mỹ kỳ hạn 10 năm tăng vọt lên mức cao nhất trong vòng một năm tiếp tục tạo áp lực lên thị trường kim loại quý trong phiên cuối tuần. Giá vàng thế giới đêm qua có lúc rơi thẳng đứng xuống vùng 1.716 USD một ounce, sau đó hồi phục dần và đóng cửa tại 1.735 USD, giảm gần 35 USD so với phiên trước. Đây là phiên giảm thứ tư trong tuần, trùng với dự đoán trước đó của nhiều chuyên gia Phố Wall.Thị trường kim loại quý trong nước cũng bị tác động mạnh bởi diễn biến thế giới. Công ty Vàng bạc đá quý Sài Gòn (

## Chuyển dữ liệu dạng text về ma trận (n x m) bằng TF-IDF

* Bạn cần viết đoạn mã tương ứng trong cell bên dưới. Theo các bước được gợi ý

In [6]:
data_train

{'data': ['Mời độc giả đặt câu hỏi tại đây\n',
  '(Nguồn và ảnh: Trung tâm cấy tóc New Hair)\n',
  'Lợi suất trái phiếu chính phủ Mỹ kỳ hạn 10 năm tăng vọt lên mức cao nhất trong vòng một năm tiếp tục tạo áp lực lên thị trường kim loại quý trong phiên cuối tuần. Giá vàng thế giới đêm qua có lúc rơi thẳng đứng xuống vùng 1.716 USD một ounce, sau đó hồi phục dần và đóng cửa tại 1.735 USD, giảm gần 35 USD so với phiên trước. Đây là phiên giảm thứ tư trong tuần, trùng với dự đoán trước đó của nhiều chuyên gia Phố Wall.Thị trường kim loại quý trong nước cũng bị tác động mạnh bởi diễn biến thế giới. Công ty Vàng bạc đá quý Sài Gòn (SJC) sáng nay mua vào 55,65 triệu đồng và bán ra 56,15 triệu đồng, lần lượt mất 300.000 đồng và 200.000 đồng so với hôm qua.Tập đoàn Vàng bạc đá quý DOJI niêm yết giá mua vào thấp hơn, chỉ 55,5 triệu đồng và bán ra 56,15 triệu đồng. Mỗi lượng vàng giao dịch tại đây giảm 200.000 đồng.Giá biến động mạnh hơn ở những cửa hàng nhỏ. Tiệm vàng Mi Hồng ở quận Bình Thạnh, 

In [7]:
with open("vietnamese_stopwords.txt",encoding="utf-8") as f:
    stopwords = f.readlines()
stopwords = [x.strip().replace(" ", "_") for x in stopwords] 
print(f"Số lượng stopwords: {len(stopwords)}")
print(stopwords[:10])

# Chuyển hoá dữ liệu text về dạng vector TF 
#     - loại bỏ từ dừng
#     - sinh từ điển
module_count_vector = CountVectorizer(stop_words=stopwords)
model_rf_preprocess = Pipeline([('vect', module_count_vector),
                    ('tfidf', TfidfTransformer()),
                    ])
# Hàm thực hiện chuyển đổi dữ liệu text thành dữ liệu số dạng ma trận 
# Input: Dữ liệu 2 chiều dạng numpy.array, mảng nhãn id dạng numpy.array 
data_preprocessed = model_rf_preprocess.fit_transform(data_train.data, data_train.target)

X = data_preprocessed # thuoc tinh
Y = data_train.target #nhan

print(f"\nSố lượng từ trong từ điển: {len(module_count_vector.vocabulary_)}")
print(f"Kích thước dữ liệu sau khi xử lý: {X.shape}")
print(f"Kích thước nhãn tương ứng: {Y.shape}")

Số lượng stopwords: 1942
['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']

Số lượng từ trong từ điển: 12797
Kích thước dữ liệu sau khi xử lý: (1339, 12797)
Kích thước nhãn tương ứng: (1339,)


In [8]:
print(X[100].toarray())
print(Y[100])

[[0.         0.         0.         ... 0.         0.14048828 0.        ]]
5


In [9]:
sum(sum(X[100].toarray() != 0))

289

In [10]:
print(X[100])

  (0, 81)	0.015211595534715633
  (0, 100)	0.020769547555053954
  (0, 156)	0.023262873797037953
  (0, 188)	0.04722498744523733
  (0, 269)	0.03324036700472501
  (0, 392)	0.024783841259467605
  (0, 397)	0.034232954192352914
  (0, 418)	0.048918066523847725
  (0, 662)	0.022769929356223215
  (0, 909)	0.033182248640039956
  (0, 1194)	0.05117031195708949
  (0, 1209)	0.10234062391417897
  (0, 1219)	0.05117031195708949
  (0, 1271)	0.016005705134410072
  (0, 1590)	0.034926623903062066
  (0, 1631)	0.023493949966261335
  (0, 1783)	0.04734509560871461
  (0, 1866)	0.05117031195708949
  (0, 2076)	0.014547704347804936
  (0, 2101)	0.02693672471116765
  (0, 2111)	0.02577563465433512
  (0, 2135)	0.04322051581452055
  (0, 2140)	0.01566196368628259
  (0, 2159)	0.01608450478874651
  (0, 2170)	0.02950839772591026
  :	:
  (0, 12272)	0.021259537682082122
  (0, 12454)	0.0758997005019029
  (0, 12510)	0.01778617457985167
  (0, 12518)	0.038832198646960935
  (0, 12522)	0.09355442947181114
  (0, 12536)	0.054883703258